<img src="images/dask_horizontal.svg"
     width="45%"
     alt="Dask logo">
     
# Delayed

What if you don't have an array or dataframe? Instead of having blocks where the function is applied to each block, you can decorate functions with `@delayed` and have the functions themselves be lazy. 

In [ ]:
import time
import random
from dask import delayed

@delayed
def inc(x):
    time.sleep(random.random())
    return x + 1

@delayed
def double(x):
    time.sleep(random.random())
    return 2 * x
  
@delayed
def add(x, y):
    time.sleep(random.random())
    return x + y 

In [ ]:
data = [1, 2, 3, 4]

output = []
for i in data:
    a = inc(i)
    b = double(i)
    c = add(a, b)
    output.append(c)

total = sum(output)

In [ ]:
total.visualize()

Of course objects can also be converted to `delayed`. Here we can convert from a `dask.array` to a `numpy.array` of delayed objects. 

In [ ]:
import dask.array as da

arr = da.random.random(size=(1_000, 1_000), chunks=(250, 500))

arr_delayed = arr.to_delayed()
arr_delayed

In [ ]:
arr_delayed[0, 1].sum().compute()

In [ ]:
arr.blocks[0, 1].sum().compute()